# 文本分类实例

## Step1 导入相关包

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [ ]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

## Step3 划分数据集

In [ ]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

## Step4 数据集预处理

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

## Step5 创建模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

In [ ]:
model.config

## Step6 创建评估函数

In [ ]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [ ]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略，每训练完一个epoch计算一次评估指标
                            #    evaluation_strategy="steps",   # 评估策略使用steps时，需要指定eval_steps，就是多少步评估一次
                            #    eval_steps=100,
                               save_strategy="epoch",           # 保存策略，每一个epoch存一次
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标，就是根据什么评估指标判断哪个模型最好
                               load_best_model_at_end=True)     # 训练完成后是否加载最优模型
train_args

## Step8 创建Trainer

In [ ]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [ ]:
trainer.train()

## Step10 模型评估

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

## Step11 模型预测

In [ ]:
trainer.predict(tokenized_datasets["test"])

In [ ]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
sen = "我觉得不错！"
pipe(sen)

# Step12 查看tensorboard

In [ ]:
# 方式一：使用vscode自带的tensorboard功能，在命令面板中输入:
# 方式二
    # 在命令行输入conda activate transformers，进入checkpoints文件夹，
    # 然后输入tensorboard --logdir runs，就能得到一个地址，打开地址就可以看到tensorboard
